### Libraries

In [56]:
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import importlib
importlib.reload(utils)
import datetime

'machine learning library imports'
from sklearn import tree
from sklearn import metrics
from sklearn.utils import shuffle

### Load data

In [57]:
AUGMENTATION = 'GReaT' # options: 'None', 'GReaT', 'SMOTE', 'SMOTE-NC' or 'RealTabFormer'

data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mbtcp.unit_id', 
                                                       'mbtcp.trans_id'])                                    

Loading complete.
Train data: 1500000 rows, 46 columns. 
Test data: 381934 rows, 46 columns.


### Data preparation

In [58]:
# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_type', 'Attack_label'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [59]:
# Encode the training and test labels if needed
X_train_enc, X_test_enc, info = utils.encode_categorical(X_train, X_test)

Categorical features to be encoded:

http.request.method
http.referer
mqtt.conack.flags
dns.qry.name.len
mqtt.protoname
mqtt.topic
http.request.version

Encoding complete.
No of features before encoding: 44
No of features after encoding: 100


#### Label encoding

In [60]:
y_train_enc, y_test_enc, le = utils.encode_labels(y_train, y_test)

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model Training

In [61]:
# Shuffle training data
X_train_enc, y_train_enc = shuffle(X_train_enc, y_train_enc, random_state=42)

# Instantiate the decision tree classifier
dt = tree.DecisionTreeClassifier(random_state=42)

# Train the model
dt_clf = dt.fit(X_train_enc, y_train_enc)

### Model Evaluation

In [62]:
predictions = dt_clf.predict(X_test_enc)
predictions[:3]

array([ 7, 11,  4])

In [63]:
# Calculate metrics
accuracy = metrics.accuracy_score(y_test_enc, predictions)
precision_w = metrics.precision_score(y_test_enc, predictions, average='weighted', zero_division=1)
recall_w = metrics.recall_score(y_test_enc, predictions, average='weighted')
f1_score_w = metrics.f1_score(y_test_enc, predictions, average='weighted')
precision_m = metrics.precision_score(y_test_enc, predictions, average='macro', zero_division=1)
recall_m = metrics.recall_score(y_test_enc, predictions, average='macro')
f1_score_m = metrics.f1_score(y_test_enc, predictions, average='macro')

In [64]:
# Create dictionary for results
results = {
    "model": "Decision Tree",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

utils.print_results_table(results)

╒══════════════════════╤═════════╕
│ Metric               │ Value   │
╞══════════════════════╪═════════╡
│ Accuracy             │ 96.56%  │
├──────────────────────┼─────────┤
│ Precision (macro)    │ 80.03%  │
├──────────────────────┼─────────┤
│ Recall (macro)       │ 87.04%  │
├──────────────────────┼─────────┤
│ F1 (macro)           │ 78.59%  │
├──────────────────────┼─────────┤
│ Precision (weighted) │ 98.09%  │
├──────────────────────┼─────────┤
│ Recall (weighted)    │ 96.56%  │
├──────────────────────┼─────────┤
│ F1 (weighted)        │ 97.25%  │
╘══════════════════════╧═════════╛


#### Save Metrics Results 

In [65]:
# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/dt.csv')

In [66]:
conf_mat = metrics.confusion_matrix(y_test_enc, predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                           index = attack_labels, 
                           columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'

# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4326,0,0,0,0,182,88,0,0,121,41,5,0,16,3
DDoS_HTTP,0,8361,0,0,0,27,0,0,0,0,0,0,0,335,905
DDoS_ICMP,1,0,13314,0,0,186,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,9124,0,0,866,0,0,19,0,0,0,0,0
DDoS_UDP,0,0,0,0,22756,0,1845,0,0,0,0,0,0,0,0
Fingerprinting,4,0,17,0,0,84,11,0,0,23,7,0,0,0,0
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,0,0,0,0,0,0,0,272776,0,0,0,0,0,0,0
Password,0,165,0,0,0,1056,144,0,7410,0,0,765,549,7,12
